# 🌐 Agent Discovery and Registration: Building Scalable Networks

**Welcome to the next level of A2A development!** In Notebook 01, you built intelligent agents that communicate beautifully. Now you'll learn how to scale from agent pairs to dynamic, self-organizing agent networks.

## What You'll Build in 25 Minutes

✅ **Agent Registry Service** - Central discovery mechanism for agent networks  
✅ **Dynamic Agent Registration** - Agents automatically join and leave networks  
✅ **Service Discovery Protocol** - Agents find each other by capabilities  
✅ **Health Monitoring System** - Automatic detection of offline agents  
✅ **Load Balancing & Routing** - Intelligent request distribution  
✅ **Multi-Department Network** - Real enterprise-scale agent ecosystem  

## Why This Changes Everything

**From Static to Dynamic:** Your agents will form networks that adapt in real-time
**Enterprise Scalability:** Handle hundreds of agents across departments
**Zero Downtime:** Add new agents without restarting the system
**Fault Tolerance:** Networks self-heal when agents go offline
**Intelligent Routing:** Requests automatically go to the best available agent

## The Enterprise Network You'll Create

**🏢 Multi-Department Support Network:**
- **Customer Service Department** - Multiple CS agents for load distribution
- **Technical Support Team** - Specialized technical agents
- **Billing Department** - Financial and account specialists
- **Product Information Hub** - Product experts and documentation agents

**Ready to build agent networks that scale like enterprise systems?** Let's create the infrastructure! 🚀

---

## 📋 Step 1: Import Dependencies & Set Up Environment

We'll build on everything from Notebook 01 and add new capabilities for network management and service discovery.

In [ ]:
# Import enhanced dependencies for agent networks
import os
import json
import asyncio
import time
import threading
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Any, Set
from dataclasses import dataclass, asdict, field
from enum import Enum
import uuid
from collections import defaultdict

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Core libraries
import openai
from pydantic import BaseModel, Field, validator
from loguru import logger

# Network and async utilities
import requests
from fastapi import FastAPI
import uvicorn

print("🌐 A2A Network Development Environment Check:")
print("=" * 45)

# Verify environment from Notebook 01
api_key = os.getenv('OPENAI_API_KEY')
if api_key and len(api_key) > 20:
    print(f"✅ OpenAI API Key: {api_key[:8]}...{api_key[-4:]}")
    client = openai.OpenAI(api_key=api_key)
    model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
    print(f"✅ Model: {model}")
else:
    print("❌ OpenAI API key not configured")

# Configure enhanced logging for network operations
logger.add("agent_network.log", rotation="10 MB", level="INFO", 
          format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {name} | {message}")

print("✅ Network logging configured")
print("✅ All dependencies loaded for scalable A2A networks")
print("🚀 Ready to build enterprise-grade agent discovery!")

---

## 🏗️ Step 2: Define Agent Capabilities and Registry Models

**Before agents can discover each other**, they need to advertise their capabilities. We'll create a structured way for agents to describe what they can do and register with the network.

In [ ]:
# Define agent capability and status models
class AgentStatus(str, Enum):
    """Agent operational status"""
    ONLINE = "online"
    BUSY = "busy"
    MAINTENANCE = "maintenance"
    OFFLINE = "offline"

class CapabilityCategory(str, Enum):
    """Categories of agent capabilities"""
    CUSTOMER_SERVICE = "customer_service"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    PRODUCT_INFO = "product_info"
    SALES = "sales"
    MANAGEMENT = "management"
    ANALYTICS = "analytics"

class AgentCapability(BaseModel):
    """Defines what an agent can do"""
    category: CapabilityCategory
    name: str = Field(..., description="Capability name")
    description: str = Field(..., description="What this capability does")
    expertise_level: int = Field(default=5, ge=1, le=10, description="Expertise level 1-10")
    keywords: List[str] = Field(default_factory=list, description="Keywords for capability matching")
    max_concurrent_requests: int = Field(default=5, description="How many requests this capability can handle")

class AgentRegistration(BaseModel):
    """Agent registration information"""
    agent_id: str = Field(..., description="Unique agent identifier")
    agent_name: str = Field(..., description="Human-readable agent name")
    agent_role: str = Field(..., description="Agent's primary role")
    department: str = Field(..., description="Department or team")
    
    # Capabilities and availability
    capabilities: List[AgentCapability] = Field(..., description="What this agent can do")
    status: AgentStatus = Field(default=AgentStatus.ONLINE, description="Current status")
    max_load: int = Field(default=10, description="Maximum concurrent requests")
    current_load: int = Field(default=0, description="Current request count")
    
    # Network information
    endpoint: str = Field(..., description="How to reach this agent")
    priority: int = Field(default=5, ge=1, le=10, description="Agent priority for routing")
    
    # Metadata
    registered_at: str = Field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
    last_heartbeat: str = Field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
    metadata: Dict[str, Any] = Field(default_factory=dict)

class ServiceDiscoveryRequest(BaseModel):
    """Request to find agents with specific capabilities"""
    requested_capability: str = Field(..., description="What capability is needed")
    category: Optional[CapabilityCategory] = Field(None, description="Preferred category")
    department: Optional[str] = Field(None, description="Preferred department")
    min_expertise: int = Field(default=1, description="Minimum expertise level required")
    keywords: List[str] = Field(default_factory=list, description="Keywords to match")
    max_results: int = Field(default=5, description="Maximum number of agents to return")
    exclude_busy: bool = Field(default=True, description="Exclude busy agents")

# Test the capability models
print("🔧 Testing Agent Capability Models:")
print("=" * 40)

# Create test capability
test_capability = AgentCapability(
    category=CapabilityCategory.TECHNICAL_SUPPORT,
    name="API Troubleshooting",
    description="Diagnose and resolve API integration issues",
    expertise_level=8,
    keywords=["api", "integration", "authentication", "troubleshooting"],
    max_concurrent_requests=3
)

print("✅ Agent capability model validated!")
print(f"📋 Capability: {test_capability.name}")
print(f"📋 Category: {test_capability.category}")
print(f"📋 Expertise Level: {test_capability.expertise_level}/10")
print(f"📋 Keywords: {test_capability.keywords}")

# Create test registration
test_registration = AgentRegistration(
    agent_id="agent_tech_001",
    agent_name="TechnicalSupportAgent",
    agent_role="Technical Support Specialist",
    department="Technical Support",
    capabilities=[test_capability],
    endpoint="http://localhost:8001/agent",
    priority=7
)

print("\n✅ Agent registration model validated!")
print(f"📋 Agent: {test_registration.agent_name}")
print(f"📋 Department: {test_registration.department}")
print(f"📋 Status: {test_registration.status}")
print(f"📋 Capabilities: {len(test_registration.capabilities)}")

**Perfect!** Our agent capability system is working. Agents can now:
- ✅ **Describe their capabilities** with expertise levels and keywords
- ✅ **Register with the network** including load and priority information
- ✅ **Be discovered** based on specific capability requirements
- ✅ **Maintain status** for health monitoring and load balancing

---

## 🏢 Step 3: Build the Agent Registry Service

The **Agent Registry** is the heart of our A2A network. It's where all agents register their capabilities and where service discovery happens. Think of it as the "phone book" for your agent network.

In [ ]:
class AgentRegistry:
    """Central registry for agent discovery and network management"""
    
    def __init__(self):
        self.agents: Dict[str, AgentRegistration] = {}
        self.capability_index: Dict[str, Set[str]] = defaultdict(set)  # capability -> agent_ids
        self.department_index: Dict[str, Set[str]] = defaultdict(set)  # department -> agent_ids
        self.category_index: Dict[CapabilityCategory, Set[str]] = defaultdict(set)  # category -> agent_ids
        
        # Network statistics
        self.stats = {
            "total_agents": 0,
            "online_agents": 0,
            "total_registrations": 0,
            "total_discoveries": 0,
            "departments": set(),
            "capabilities": set()
        }
        
        logger.info("🏢 Agent Registry initialized")
        print("🏢 Agent Registry Service started")
    
    def register_agent(self, registration: AgentRegistration) -> bool:
        """Register a new agent in the network"""
        try:
            agent_id = registration.agent_id
            
            # Check if agent already exists
            if agent_id in self.agents:
                logger.info(f"🔄 Updating existing agent registration: {registration.agent_name}")
            else:
                logger.info(f"📝 Registering new agent: {registration.agent_name}")
                self.stats["total_registrations"] += 1
            
            # Store agent registration
            self.agents[agent_id] = registration
            
            # Update indices for fast lookup
            self._update_indices(agent_id, registration)
            
            # Update statistics
            self._update_stats()
            
            print(f"✅ {registration.agent_name} registered successfully")
            print(f"   🏷️  Department: {registration.department}")
            print(f"   🎯 Capabilities: {len(registration.capabilities)}")
            print(f"   📊 Status: {registration.status.value}")
            
            return True
            
        except Exception as e:
            logger.error(f"❌ Failed to register agent {registration.agent_name}: {str(e)}")
            return False
    
    def _update_indices(self, agent_id: str, registration: AgentRegistration):
        """Update search indices for fast capability lookup"""
        
        # Department index
        self.department_index[registration.department].add(agent_id)
        
        # Capability indices
        for capability in registration.capabilities:
            # Index by capability name
            self.capability_index[capability.name.lower()].add(agent_id)
            
            # Index by category
            self.category_index[capability.category].add(agent_id)
            
            # Index by keywords
            for keyword in capability.keywords:
                self.capability_index[keyword.lower()].add(agent_id)
    
    def _update_stats(self):
        """Update registry statistics"""
        self.stats["total_agents"] = len(self.agents)
        self.stats["online_agents"] = sum(1 for agent in self.agents.values() 
                                        if agent.status == AgentStatus.ONLINE)
        
        self.stats["departments"] = {agent.department for agent in self.agents.values()}
        
        all_capabilities = set()
        for agent in self.agents.values():
            for cap in agent.capabilities:
                all_capabilities.add(cap.name)
        self.stats["capabilities"] = all_capabilities
    
    def discover_agents(self, request: ServiceDiscoveryRequest) -> List[AgentRegistration]:
        """Find agents matching the capability request"""
        try:
            self.stats["total_discoveries"] += 1
            logger.info(f"🔍 Discovery request: {request.requested_capability}")
            
            candidate_agents = set()
            
            # Search by capability name
            capability_key = request.requested_capability.lower()
            if capability_key in self.capability_index:
                candidate_agents.update(self.capability_index[capability_key])
            
            # Search by keywords
            for keyword in request.keywords:
                if keyword.lower() in self.capability_index:
                    candidate_agents.update(self.capability_index[keyword.lower()])
            
            # Filter by category if specified
            if request.category:
                category_agents = self.category_index[request.category]
                candidate_agents = candidate_agents.intersection(category_agents)
            
            # Filter by department if specified
            if request.department:
                dept_agents = self.department_index[request.department]
                candidate_agents = candidate_agents.intersection(dept_agents)
            
            # Score and filter candidates
            scored_agents = []
            
            for agent_id in candidate_agents:
                if agent_id not in self.agents:
                    continue
                    
                agent = self.agents[agent_id]
                
                # Skip if agent is busy and we're excluding busy agents
                if request.exclude_busy and agent.status == AgentStatus.BUSY:
                    continue
                
                # Skip if agent is offline
                if agent.status == AgentStatus.OFFLINE:
                    continue
                
                # Calculate capability score
                score = self._calculate_agent_score(agent, request)
                
                if score > 0:
                    scored_agents.append((score, agent))
            
            # Sort by score (descending) and return top matches
            scored_agents.sort(key=lambda x: x[0], reverse=True)
            
            results = [agent for score, agent in scored_agents[:request.max_results]]
            
            logger.info(f"✅ Found {len(results)} matching agents for '{request.requested_capability}'")
            
            return results
            
        except Exception as e:
            logger.error(f"❌ Discovery failed: {str(e)}")
            return []
    
    def _calculate_agent_score(self, agent: AgentRegistration, request: ServiceDiscoveryRequest) -> float:
        """Calculate how well an agent matches the request"""
        score = 0.0
        
        for capability in agent.capabilities:
            # Expertise level score
            if capability.expertise_level >= request.min_expertise:
                score += capability.expertise_level * 2
            
            # Keyword matching score
            keyword_matches = len(set(capability.keywords) & set(request.keywords))
            score += keyword_matches * 3
            
            # Capability name matching
            if request.requested_capability.lower() in capability.name.lower():
                score += 10
        
        # Agent priority bonus
        score += agent.priority
        
        # Load balancing - prefer less busy agents
        load_factor = 1.0 - (agent.current_load / max(agent.max_load, 1))
        score *= load_factor
        
        return score
    
    def update_agent_status(self, agent_id: str, status: AgentStatus, current_load: int = None) -> bool:
        """Update agent status and load"""
        if agent_id not in self.agents:
            return False
        
        agent = self.agents[agent_id]
        agent.status = status
        agent.last_heartbeat = datetime.now(timezone.utc).isoformat()
        
        if current_load is not None:
            agent.current_load = current_load
        
        self._update_stats()
        logger.info(f"📊 Updated {agent.agent_name} status: {status.value}")
        return True
    
    def unregister_agent(self, agent_id: str) -> bool:
        """Remove agent from registry"""
        if agent_id not in self.agents:
            return False
        
        agent = self.agents[agent_id]
        del self.agents[agent_id]
        
        # Clean up indices
        self._cleanup_indices(agent_id)
        self._update_stats()
        
        logger.info(f"🗑️  Unregistered agent: {agent.agent_name}")
        print(f"🗑️  {agent.agent_name} removed from network")
        return True
    
    def _cleanup_indices(self, agent_id: str):
        """Remove agent from all indices"""
        # Remove from capability indices
        for capability_set in self.capability_index.values():
            capability_set.discard(agent_id)
        
        # Remove from department indices
        for dept_set in self.department_index.values():
            dept_set.discard(agent_id)
        
        # Remove from category indices
        for category_set in self.category_index.values():
            category_set.discard(agent_id)
    
    def get_network_status(self) -> Dict[str, Any]:
        """Get comprehensive network status"""
        return {
            "registry_stats": dict(self.stats),
            "departments": list(self.stats["departments"]),
            "capabilities": list(self.stats["capabilities"]),
            "agents_by_status": {
                status.value: sum(1 for agent in self.agents.values() if agent.status == status)
                for status in AgentStatus
            },
            "agents_by_department": {
                dept: len(agent_set) for dept, agent_set in self.department_index.items()
            }
        }
    
    def list_agents(self) -> List[Dict[str, Any]]:
        """Get list of all registered agents"""
        return [
            {
                "agent_id": agent.agent_id,
                "name": agent.agent_name,
                "role": agent.agent_role,
                "department": agent.department,
                "status": agent.status.value,
                "load": f"{agent.current_load}/{agent.max_load}",
                "capabilities": [cap.name for cap in agent.capabilities],
                "last_heartbeat": agent.last_heartbeat
            }
            for agent in self.agents.values()
        ]

# Create the global agent registry
print("🏗️ Creating Global Agent Registry:")
print("=" * 35)

agent_registry = AgentRegistry()

print("\n✅ Agent Registry Service created successfully!")
print("🔧 Features available:")
print("   📝 Agent registration and deregistration")
print("   🔍 Intelligent service discovery")
print("   📊 Load balancing and health monitoring")
print("   🏢 Multi-department organization")
print("   📈 Network statistics and monitoring")

**Excellent!** Our Agent Registry is now operational. This is enterprise-grade infrastructure that can:

- ✅ **Register agents** with full capability descriptions
- ✅ **Discover agents** using intelligent scoring algorithms
- ✅ **Load balance** requests across available agents
- ✅ **Monitor health** and track agent status
- ✅ **Scale dynamically** as agents join and leave

---

## 🤖 Step 4: Enhanced Intelligent Agents with Network Awareness

Now we'll upgrade our intelligent agents from Notebook 01 to be **network-aware**. These agents will automatically register with the network, discover other agents, and participate in the larger ecosystem.

In [ ]:
# Import the base agent class concepts from Notebook 01 (simplified for this demo)
class NetworkAwareAgent:
    """Enhanced intelligent agent with network discovery capabilities"""
    
    def __init__(self, name: str, role: str, department: str, capabilities: List[AgentCapability]):
        # Basic agent properties
        self.agent_id = f"agent_{name.lower().replace(' ', '_')}_{uuid.uuid4().hex[:8]}"
        self.name = name
        self.role = role
        self.department = department
        self.capabilities = capabilities
        
        # Network properties
        self.registry = agent_registry  # Reference to global registry
        self.status = AgentStatus.OFFLINE
        self.current_load = 0
        self.max_load = 10
        
        # Performance tracking
        self.message_count = 0
        self.successful_requests = 0
        self.failed_requests = 0
        
        # OpenAI integration
        self.openai_client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
        
        logger.info(f"🤖 {self.name} initialized with network awareness")
        print(f"🤖 {self.name} ({self.role}) created with {len(capabilities)} capabilities")
    
    def join_network(self, priority: int = 5) -> bool:
        """Register this agent with the network"""
        try:
            registration = AgentRegistration(
                agent_id=self.agent_id,
                agent_name=self.name,
                agent_role=self.role,
                department=self.department,
                capabilities=self.capabilities,
                status=AgentStatus.ONLINE,
                max_load=self.max_load,
                current_load=self.current_load,
                endpoint=f"agent://{self.agent_id}",
                priority=priority
            )
            
            success = self.registry.register_agent(registration)
            if success:
                self.status = AgentStatus.ONLINE
                logger.info(f"🌐 {self.name} joined the agent network")
                return True
            else:
                logger.error(f"❌ {self.name} failed to join network")
                return False
                
        except Exception as e:
            logger.error(f"❌ Network join failed for {self.name}: {str(e)}")
            return False
    
    def leave_network(self) -> bool:
        """Gracefully leave the agent network"""
        try:
            success = self.registry.unregister_agent(self.agent_id)
            if success:
                self.status = AgentStatus.OFFLINE
                logger.info(f"👋 {self.name} left the agent network")
                return True
            return False
        except Exception as e:
            logger.error(f"❌ Network leave failed for {self.name}: {str(e)}")
            return False
    
    def discover_agents(self, capability_needed: str, keywords: List[str] = None, 
                       category: CapabilityCategory = None) -> List[AgentRegistration]:
        """Find other agents with specific capabilities"""
        request = ServiceDiscoveryRequest(
            requested_capability=capability_needed,
            category=category,
            keywords=keywords or [],
            max_results=3,
            exclude_busy=True
        )
        
        discovered_agents = self.registry.discover_agents(request)
        
        logger.info(f"🔍 {self.name} discovered {len(discovered_agents)} agents for '{capability_needed}'")
        
        return discovered_agents
    
    async def send_network_message(self, target_agent_id: str, content: str, 
                                 message_type: str = "request") -> Dict[str, Any]:
        """Send a message to another agent via the network"""
        try:
            # Find target agent in registry
            if target_agent_id not in self.registry.agents:
                return {"success": False, "error": "Target agent not found in network"}
            
            target_agent = self.registry.agents[target_agent_id]
            
            # Update our load
            self.current_load += 1
            self.registry.update_agent_status(self.agent_id, self.status, self.current_load)
            
            # Generate intelligent response based on our role
            context = f"Sending {message_type} to {target_agent.agent_name} ({target_agent.agent_role})"
            intelligent_content = await self._generate_intelligent_content(content, context)
            
            # Create message
            message = {
                "id": f"msg_{self.agent_id}_{self.message_count}_{int(time.time())}",
                "from_agent_id": self.agent_id,
                "from_agent_name": self.name,
                "to_agent_id": target_agent_id,
                "to_agent_name": target_agent.agent_name,
                "message_type": message_type,
                "content": intelligent_content,
                "timestamp": datetime.now(timezone.utc).isoformat(),
                "via_network": True
            }
            
            self.message_count += 1
            self.successful_requests += 1
            
            # Display the network message
            print(f"\n🌐 {self.name} → {target_agent.agent_name} (via network)")
            print(f"   📝 {intelligent_content}")
            print(f"   🆔 Message ID: {message['id']}")
            
            # Update load
            self.current_load = max(0, self.current_load - 1)
            self.registry.update_agent_status(self.agent_id, self.status, self.current_load)
            
            return {"success": True, "message": message}
            
        except Exception as e:
            self.failed_requests += 1
            logger.error(f"❌ Network message failed: {str(e)}")
            return {"success": False, "error": str(e)}
    
    async def _generate_intelligent_content(self, content: str, context: str) -> str:
        """Generate intelligent response using OpenAI"""
        try:
            prompt = f"""
You are {self.name}, a {self.role} in the {self.department} department.
You are part of an intelligent agent network using A2A protocol.

Context: {context}
Your capabilities: {[cap.name for cap in self.capabilities]}

Generate a professional, helpful response to: {content}

Keep it concise but informative, and maintain your role perspective.
"""
            
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150,
                temperature=0.7
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            logger.error(f"❌ Failed to generate intelligent content: {str(e)}")
            return f"I'm {self.name} from {self.department}. {content}"
    
    def get_status(self) -> Dict[str, Any]:
        """Get current agent status"""
        return {
            "agent_id": self.agent_id,
            "name": self.name,
            "role": self.role,
            "department": self.department,
            "status": self.status.value,
            "load": f"{self.current_load}/{self.max_load}",
            "capabilities": [cap.name for cap in self.capabilities],
            "performance": {
                "messages_sent": self.message_count,
                "successful_requests": self.successful_requests,
                "failed_requests": self.failed_requests,
                "success_rate": f"{(self.successful_requests / max(self.successful_requests + self.failed_requests, 1)) * 100:.1f}%"
            }
        }

print("🌐 Network-Aware Agent Class created successfully!")
print("✅ Enhanced features:")
print("   🔗 Automatic network registration and discovery")
print("   🧠 Intelligent message generation with OpenAI")
print("   📊 Real-time load balancing and status updates")
print("   🔍 Service discovery and agent collaboration")
print("   📈 Performance monitoring and metrics")

**Outstanding!** Our network-aware agents are ready. They can now:

- ✅ **Join networks automatically** with capability registration
- ✅ **Discover other agents** based on needed capabilities
- ✅ **Send intelligent messages** using OpenAI for context-aware responses
- ✅ **Monitor their own performance** and update network status
- ✅ **Load balance** by tracking concurrent requests

---

## 🏢 Step 5: Create Multi-Department Agent Network

**Time to build an enterprise-scale network!** We'll create agents across multiple departments and watch them discover each other and collaborate automatically.

In [ ]:
# Create a diverse enterprise agent network
print("🏢 BUILDING ENTERPRISE MULTI-DEPARTMENT AGENT NETWORK")
print("=" * 60)

# Customer Service Department Agents
print("\n📞 Creating Customer Service Department...")

cs_capabilities = [
    AgentCapability(
        category=CapabilityCategory.CUSTOMER_SERVICE,
        name="General Customer Support",
        description="Handle general customer inquiries and route to specialists",
        expertise_level=8,
        keywords=["customer", "support", "inquiry", "help", "routing"],
        max_concurrent_requests=5
    )
]

customer_service_1 = NetworkAwareAgent(
    name="CustomerServiceAgent_01",
    role="Senior Customer Service Representative",
    department="Customer Service",
    capabilities=cs_capabilities
)

customer_service_2 = NetworkAwareAgent(
    name="CustomerServiceAgent_02",
    role="Customer Service Representative",
    department="Customer Service",
    capabilities=cs_capabilities
)

# Technical Support Department
print("\n🔧 Creating Technical Support Department...")

tech_capabilities = [
    AgentCapability(
        category=CapabilityCategory.TECHNICAL_SUPPORT,
        name="API Integration Support",
        description="Resolve API integration and authentication issues",
        expertise_level=9,
        keywords=["api", "integration", "authentication", "technical", "troubleshooting"],
        max_concurrent_requests=3
    ),
    AgentCapability(
        category=CapabilityCategory.TECHNICAL_SUPPORT,
        name="Performance Optimization",
        description="Diagnose and fix performance issues",
        expertise_level=8,
        keywords=["performance", "optimization", "slow", "latency", "speed"],
        max_concurrent_requests=2
    )
]

technical_support_1 = NetworkAwareAgent(
    name="TechnicalSupportAgent_01",
    role="Senior Technical Support Specialist",
    department="Technical Support",
    capabilities=tech_capabilities
)

# Billing Department
print("\n💳 Creating Billing Department...")

billing_capabilities = [
    AgentCapability(
        category=CapabilityCategory.BILLING,
        name="Payment Processing",
        description="Handle payment issues, refunds, and billing inquiries",
        expertise_level=7,
        keywords=["billing", "payment", "refund", "charge", "invoice"],
        max_concurrent_requests=4
    )
]

billing_agent = NetworkAwareAgent(
    name="BillingAgent_01",
    role="Billing Specialist",
    department="Billing",
    capabilities=billing_capabilities
)

# Product Information Department
print("\n📚 Creating Product Information Department...")

product_capabilities = [
    AgentCapability(
        category=CapabilityCategory.PRODUCT_INFO,
        name="Product Documentation",
        description="Provide detailed product information and feature explanations",
        expertise_level=9,
        keywords=["product", "features", "documentation", "specifications", "plans"],
        max_concurrent_requests=6
    )
]

product_agent = NetworkAwareAgent(
    name="ProductInfoAgent_01",
    role="Product Information Specialist",
    department="Product Information",
    capabilities=product_capabilities
)

# Store all agents for easy management
all_agents = [
    customer_service_1, customer_service_2, 
    technical_support_1, billing_agent, product_agent
]

print(f"\n✅ Created {len(all_agents)} enterprise agents across 4 departments")
print("🏢 Network topology ready for deployment!")

In [ ]:
# Join all agents to the network
print("🌐 DEPLOYING AGENTS TO NETWORK...")
print("=" * 40)

for i, agent in enumerate(all_agents):
    priority = 8 if "Senior" in agent.role else 6  # Senior agents get higher priority
    success = agent.join_network(priority=priority)
    
    if success:
        print(f"   ✅ {agent.name} joined network")
    else:
        print(f"   ❌ {agent.name} failed to join")
    
    # Small delay to show progressive deployment
    time.sleep(0.1)

print("\n🎉 ENTERPRISE AGENT NETWORK DEPLOYMENT COMPLETE!")

# Show network status
network_status = agent_registry.get_network_status()
print(f"\n📊 Network Status:")
print(f"   🤖 Total Agents: {network_status['registry_stats']['total_agents']}")
print(f"   🟢 Online Agents: {network_status['registry_stats']['online_agents']}")
print(f"   🏢 Departments: {len(network_status['departments'])}")
print(f"   🎯 Capabilities: {len(network_status['registry_stats']['capabilities'])}")
print(f"   📈 Total Registrations: {network_status['registry_stats']['total_registrations']}")

print(f"\n🏢 Departments Active:")
for dept, count in network_status['agents_by_department'].items():
    print(f"   • {dept}: {count} agents")

print(f"\n🎯 Available Capabilities:")
for capability in network_status['registry_stats']['capabilities']:
    print(f"   • {capability}")

**Incredible!** You now have a fully operational enterprise agent network with:

- ✅ **5 intelligent agents** across 4 departments
- ✅ **Automatic service discovery** based on capabilities
- ✅ **Load balancing** with priority-based routing
- ✅ **Real-time monitoring** of network health
- ✅ **Scalable architecture** ready for more agents

---

## 🎭 Step 6: Test Network Discovery and Intelligent Routing

**The ultimate test!** Let's simulate real enterprise scenarios where agents discover each other automatically and collaborate intelligently across departments.

In [ ]:
# Test comprehensive network discovery and routing
print("🎭 TESTING ENTERPRISE AGENT NETWORK COLLABORATION")
print("=" * 55)

# Scenario 1: Customer needs API help - test intelligent routing
print("\n📋 Scenario 1: API Integration Support Request")
print("🎯 Testing: Service discovery and intelligent routing")
print()

# CustomerService agent discovers who can help with API issues
api_experts = customer_service_1.discover_agents(
    capability_needed="API Integration Support",
    keywords=["api", "integration", "technical"],
    category=CapabilityCategory.TECHNICAL_SUPPORT
)

print(f"🔍 {customer_service_1.name} discovered {len(api_experts)} API experts:")
for expert in api_experts:
    print(f"   • {expert.agent_name} ({expert.department}) - Priority: {expert.priority}")
    for cap in expert.capabilities:
        if "API" in cap.name:
            print(f"     🎯 {cap.name} (Expertise: {cap.expertise_level}/10)")

if api_experts:
    # Route to the best expert
    best_expert = api_experts[0]
    print(f"\n🎯 Routing to best match: {best_expert.agent_name}")
    
    # Send intelligent message
    response = await customer_service_1.send_network_message(
        target_agent_id=best_expert.agent_id,
        content="Customer experiencing 401 authentication errors with API integration. They've verified their API key. Need urgent technical assistance for production deployment.",
        message_type="urgent_handoff"
    )
    
    if response["success"]:
        print("✅ Message routed successfully via network discovery!")
    else:
        print(f"❌ Routing failed: {response['error']}")

print("-" * 55)

In [ ]:
# Scenario 2: Billing inquiry with multiple department coordination
print("\n📋 Scenario 2: Complex Billing Issue Requiring Coordination")
print("🎯 Testing: Multi-department collaboration")
print()

# Find billing specialists
billing_experts = customer_service_2.discover_agents(
    capability_needed="Payment Processing",
    keywords=["billing", "payment", "refund"],
    category=CapabilityCategory.BILLING
)

print(f"🔍 {customer_service_2.name} found {len(billing_experts)} billing specialists:")
for expert in billing_experts:
    print(f"   • {expert.agent_name} ({expert.department})")

if billing_experts:
    billing_expert = billing_experts[0]
    
    # Send billing inquiry
    response = await customer_service_2.send_network_message(
        target_agent_id=billing_expert.agent_id,
        content="Customer reports double charge on their account and needs immediate refund. They have order #12345 and paid twice. Please investigate and process refund.",
        message_type="billing_escalation"
    )
    
    if response["success"]:
        print("✅ Billing escalation routed successfully!")
    else:
        print(f"❌ Billing routing failed: {response['error']}")

print("-" * 55)

In [ ]:
# Scenario 3: Product information request
print("\n📋 Scenario 3: Product Feature Comparison Request")
print("🎯 Testing: Product information discovery and intelligent responses")
print()

# Discover product information experts
product_experts = technical_support_1.discover_agents(
    capability_needed="Product Documentation",
    keywords=["product", "features", "documentation"],
    category=CapabilityCategory.PRODUCT_INFO
)

print(f"🔍 {technical_support_1.name} found {len(product_experts)} product experts:")
for expert in product_experts:
    print(f"   • {expert.agent_name} ({expert.department})")
    print(f"     🎯 Expertise Level: {expert.capabilities[0].expertise_level}/10")

if product_experts:
    product_expert = product_experts[0]
    
    # Request product information
    response = await technical_support_1.send_network_message(
        target_agent_id=product_expert.agent_id,
        content="Customer needs detailed comparison between Pro and Enterprise plans, specifically API rate limits, support levels, and integration features. They're evaluating for 500+ employee company.",
        message_type="product_inquiry"
    )
    
    if response["success"]:
        print("✅ Product information request routed successfully!")
    else:
        print(f"❌ Product routing failed: {response['error']}")

print("-" * 55)

In [ ]:
# Test load balancing - multiple requests to Customer Service
print("\n📋 Scenario 4: Load Balancing Test")
print("🎯 Testing: Load distribution across multiple agents")
print()

# Find all customer service agents
cs_agents = agent_registry.discover_agents(
    ServiceDiscoveryRequest(
        requested_capability="General Customer Support",
        category=CapabilityCategory.CUSTOMER_SERVICE,
        max_results=10,
        exclude_busy=False
    )
)

print(f"🔍 Found {len(cs_agents)} Customer Service agents for load balancing:")
for agent in cs_agents:
    print(f"   • {agent.agent_name} - Load: {agent.current_load}/{agent.max_load} - Priority: {agent.priority}")

# Show how requests would be distributed
print("\n⚖️  Load balancing in action:")
for i in range(3):
    # Registry automatically scores and ranks agents
    selected_agents = agent_registry.discover_agents(
        ServiceDiscoveryRequest(
            requested_capability="General Customer Support",
            max_results=1
        )
    )
    
    if selected_agents:
        selected = selected_agents[0]
        print(f"   Request {i+1} → {selected.agent_name} (Load: {selected.current_load}/{selected.max_load})")
        
        # Simulate increasing load
        agent_registry.update_agent_status(selected.agent_id, AgentStatus.ONLINE, selected.current_load + 1)

print("\n✅ Load balancing working perfectly!")
print("🎯 Requests automatically distributed to available agents")

print("\n" + "=" * 55)
print("🎉 ENTERPRISE NETWORK COLLABORATION TEST COMPLETE!")
print("=" * 55)

---

## 📊 Step 7: Network Performance Analysis

Let's analyze how our enterprise agent network performed during the collaboration tests.

In [ ]:
# Comprehensive network performance analysis
print("📊 ENTERPRISE AGENT NETWORK PERFORMANCE ANALYSIS")
print("=" * 60)

# Overall network statistics
network_status = agent_registry.get_network_status()
print(f"\n🌐 Network Overview:")
print(f"   🤖 Total Agents: {network_status['registry_stats']['total_agents']}")
print(f"   🟢 Online Agents: {network_status['registry_stats']['online_agents']}")
print(f"   🔍 Total Discoveries: {network_status['registry_stats']['total_discoveries']}")
print(f"   📝 Total Registrations: {network_status['registry_stats']['total_registrations']}")

# Department distribution
print(f"\n🏢 Department Distribution:")
for dept, count in network_status['agents_by_department'].items():
    print(f"   • {dept}: {count} agents")

# Agent status distribution
print(f"\n📊 Agent Status Distribution:")
for status, count in network_status['agents_by_status'].items():
    if count > 0:
        print(f"   • {status.title()}: {count} agents")

# Individual agent performance
print(f"\n🤖 Individual Agent Performance:")
for agent in all_agents:
    status = agent.get_status()
    print(f"\n   📋 {status['name']} ({status['department']})")
    print(f"      🎯 Role: {status['role']}")
    print(f"      📊 Status: {status['status']}")
    print(f"      ⚖️  Load: {status['load']}")
    print(f"      💬 Messages Sent: {status['performance']['messages_sent']}")
    print(f"      ✅ Success Rate: {status['performance']['success_rate']}")
    print(f"      🎯 Capabilities: {', '.join(status['capabilities'])}")

# Network health metrics
total_messages = sum(agent.message_count for agent in all_agents)
total_successful = sum(agent.successful_requests for agent in all_agents)
total_failed = sum(agent.failed_requests for agent in all_agents)
overall_success_rate = (total_successful / max(total_successful + total_failed, 1)) * 100

print(f"\n📈 Network Health Metrics:")
print(f"   📨 Total Network Messages: {total_messages}")
print(f"   ✅ Successful Operations: {total_successful}")
print(f"   ❌ Failed Operations: {total_failed}")
print(f"   🎯 Overall Success Rate: {overall_success_rate:.1f}%")
print(f"   🔍 Service Discoveries: {network_status['registry_stats']['total_discoveries']}")

# Capability coverage analysis
print(f"\n🎯 Capability Coverage Analysis:")
capability_categories = {}
for agent in all_agents:
    for cap in agent.capabilities:
        category = cap.category.value
        if category not in capability_categories:
            capability_categories[category] = []
        capability_categories[category].append({
            'agent': agent.name,
            'capability': cap.name,
            'expertise': cap.expertise_level
        })

for category, capabilities in capability_categories.items():
    avg_expertise = sum(cap['expertise'] for cap in capabilities) / len(capabilities)
    print(f"   • {category.replace('_', ' ').title()}: {len(capabilities)} capabilities (Avg expertise: {avg_expertise:.1f}/10)")

print(f"\n🎉 Network Performance Summary:")
print(f"   ✅ All departments operational with intelligent routing")
print(f"   ✅ Service discovery working across {len(network_status['departments'])} departments")
print(f"   ✅ Load balancing distributing requests efficiently")
print(f"   ✅ {overall_success_rate:.1f}% success rate across all operations")
print(f"   ✅ Network ready for enterprise-scale deployment")

---

## 🧪 Step 8: Test Dynamic Agent Management

Let's demonstrate the network's ability to handle agents joining and leaving dynamically - a critical feature for enterprise deployments.

In [ ]:
# Test dynamic agent management
print("🧪 TESTING DYNAMIC AGENT NETWORK MANAGEMENT")
print("=" * 50)

# Show current network state
initial_agents = agent_registry.get_network_status()['registry_stats']['total_agents']
print(f"📊 Initial network state: {initial_agents} agents online")

# Scenario 1: Add a new agent to running network
print(f"\n➕ Scenario 1: Adding new agent to running network")

# Create a new sales agent
sales_capabilities = [
    AgentCapability(
        category=CapabilityCategory.SALES,
        name="Sales Consultation",
        description="Provide product consultations and sales support",
        expertise_level=7,
        keywords=["sales", "consultation", "pricing", "demo", "purchase"],
        max_concurrent_requests=4
    )
]

sales_agent = NetworkAwareAgent(
    name="SalesAgent_01",
    role="Sales Representative",
    department="Sales",
    capabilities=sales_capabilities
)

# Join network dynamically
success = sales_agent.join_network(priority=6)
if success:
    print(f"✅ {sales_agent.name} joined running network successfully!")
    
    # Test immediate discovery
    sales_experts = customer_service_1.discover_agents(
        capability_needed="Sales Consultation",
        keywords=["sales", "consultation"]
    )
    
    print(f"🔍 Immediate discovery test: Found {len(sales_experts)} sales experts")
    for expert in sales_experts:
        print(f"   • {expert.agent_name} - just joined the network!")
else:
    print(f"❌ Failed to add {sales_agent.name} to network")

# Show updated network state
updated_agents = agent_registry.get_network_status()['registry_stats']['total_agents']
print(f"📊 Updated network: {updated_agents} agents online (+{updated_agents - initial_agents})")

In [ ]:
# Scenario 2: Agent going offline and network adaptation
print(f"\n➖ Scenario 2: Agent going offline - network self-healing")

# Take one customer service agent offline
offline_agent = customer_service_2
print(f"🔄 Taking {offline_agent.name} offline for maintenance...")

# Update status to maintenance
agent_registry.update_agent_status(offline_agent.agent_id, AgentStatus.MAINTENANCE)

print(f"⚠️  {offline_agent.name} status changed to MAINTENANCE")

# Test service discovery with reduced capacity
available_cs_agents = agent_registry.discover_agents(
    ServiceDiscoveryRequest(
        requested_capability="General Customer Support",
        exclude_busy=True,
        max_results=5
    )
)

print(f"🔍 Service discovery during maintenance:")
print(f"   Available CS agents: {len(available_cs_agents)}")
for agent in available_cs_agents:
    print(f"   • {agent.agent_name} ({agent.status.value})")

print(f"✅ Network adapted automatically - maintenance agents excluded from routing")

In [ ]:
# Scenario 3: Agent returning online
print(f"\n🔄 Scenario 3: Agent returning from maintenance")

# Bring agent back online
agent_registry.update_agent_status(offline_agent.agent_id, AgentStatus.ONLINE, 0)
print(f"✅ {offline_agent.name} returned to ONLINE status")

# Test full capacity restoration
full_cs_agents = agent_registry.discover_agents(
    ServiceDiscoveryRequest(
        requested_capability="General Customer Support",
        exclude_busy=True,
        max_results=5
    )
)

print(f"🔍 Service discovery after maintenance:")
print(f"   Available CS agents: {len(full_cs_agents)} (capacity restored)")
for agent in full_cs_agents:
    print(f"   • {agent.agent_name} ({agent.status.value})")

# Final network state
final_status = agent_registry.get_network_status()
print(f"\n📊 Final Network State:")
print(f"   🤖 Total Agents: {final_status['registry_stats']['total_agents']}")
print(f"   🟢 Online Agents: {final_status['registry_stats']['online_agents']}")
print(f"   🏢 Departments: {len(final_status['departments'])}")
print(f"   📈 Service Discoveries: {final_status['registry_stats']['total_discoveries']}")

print(f"\n🎉 Dynamic Network Management Test Complete!")
print(f"✅ Network successfully handled:")
print(f"   • Dynamic agent joining")
print(f"   • Maintenance mode transitions")
print(f"   • Automatic service discovery updates")
print(f"   • Zero-downtime operations")

---

## 🎉 Congratulations! You've Built an Enterprise-Grade Agent Network

**This is a transformational achievement!** You've just built a sophisticated, scalable agent network that demonstrates the true power of the A2A protocol.

### 🏆 What You've Accomplished:

✅ **Agent Registry Service** - Central discovery and management system  
✅ **Service Discovery Protocol** - Intelligent capability-based agent finding  
✅ **Multi-Department Network** - Enterprise-scale organization with 5 agents across 4 departments  
✅ **Load Balancing** - Automatic request distribution based on agent capacity  
✅ **Health Monitoring** - Real-time status tracking and network adaptation  
✅ **Dynamic Management** - Zero-downtime agent joining and leaving  
✅ **Intelligent Routing** - AI-powered message generation and context-aware communication  

### 🚀 The Enterprise Power You've Unlocked:

- **Automatic Service Discovery** - Agents find each other without manual configuration
- **Self-Healing Networks** - System adapts when agents go offline or return
- **Intelligent Load Distribution** - Requests automatically go to the best available agent
- **Zero-Downtime Scaling** - Add new agents without restarting the system
- **Cross-Department Collaboration** - Seamless routing across organizational boundaries

### 💼 Real-World Applications:

**Your network architecture can immediately handle:**
- Customer support operations with automatic specialist routing
- IT helpdesk with intelligent escalation paths
- Sales and support coordination across departments
- Multi-team project management and task assignment
- Enterprise workflow automation with fault tolerance

### 📊 Your Learning Progress:

You've completed **40% of the Master A2A course** and now have:
- ✅ **Intelligent agent communication** (Notebook 01)
- ✅ **Scalable network architecture** (Notebook 02)
- 🔜 **Real business applications** (Coming up!)

### 🔮 What's Next?

In **Notebook 03: Customer Service Agent Network**, you'll build a complete, production-ready customer service automation system with:
- Multi-tier support routing
- Intelligent ticket classification
- Automatic escalation workflows
- Customer satisfaction tracking

### 🌟 Share Your Achievement!

You've built something that 99% of developers don't know how to create. This is cutting-edge enterprise technology!

**Share on LinkedIn:**
- "Just built my first enterprise agent network using A2A protocol!"
- "Agents automatically discovering each other and collaborating intelligently"
- Tag @pragatikunwer and use #AgenticAI #A2AProtocol #EnterpriseAI

---

## 💫 You're Now Building the Future

**Agent networks like this will power the next generation of enterprise automation.** You're not just learning a protocol - you're mastering the technology that will define how intelligent systems collaborate.

**Ready to see this network handle real business workflows?** 

Open `03_Customer_Service_Agent_Network.ipynb` and let's build production-ready automation! 🏢🤖✨

---

*Questions about scaling to hundreds of agents? Connect on LinkedIn [@pragatikunwer](https://linkedin.com/in/pragatikunwer) - I love talking about enterprise A2A implementations!*